In [1]:
import pandas as pd


df = pd.read_parquet("./yellow_tripdata_2023-01.parquet")
print("Number of columns in January 2023 Yellow Taxi data:", df.shape[1])


Number of columns in January 2023 Yellow Taxi data: 19


In [4]:
df.head

<bound method NDFrame.head of          VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1               2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2               2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3               1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4               2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   
...           ...                  ...                   ...              ...   
3066761         2  2023-01-31 23:58:34   2023-02-01 00:12:33              NaN   
3066762         2  2023-01-31 23:31:09   2023-01-31 23:50:36              NaN   
3066763         2  2023-01-31 23:01:05   2023-01-31 23:25:36              NaN   
3066764         2  2023-01-31 23:40:00   2023-01-31 23:53:00              NaN   
3066765         2  2023-01-31 23:07:32   2023-01-31 23:21:56              NaN  

In [2]:
# Convert pickup and dropoff columns to datetime
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

# Compute trip duration in minutes
df["duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60

# Filter out outliers and invalid trips
#df = df[(df["duration_min"] > 0) & (df["duration_min"] <= 180)]

# Calculate standard deviation
std_duration = df["duration_min"].std()
print("Standard deviation of trip duration (in minutes):", std_duration)

Standard deviation of trip duration (in minutes): 42.594351241920904


In [3]:
# Convert to datetime
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

# Compute duration in minutes
df["duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60

# Total number of rows before filtering
total_rows = len(df)

# Keep only 1 to 60 minutes duration
filtered_df = df[(df["duration_min"] >= 1) & (df["duration_min"] <= 60)]

# Number of rows after filtering
filtered_rows = len(filtered_df)

# Compute fraction
fraction_remaining = filtered_rows / total_rows
print("Fraction of records with duration between 1 and 60 minutes:", round(fraction_remaining, 3))


Fraction of records with duration between 1 and 60 minutes: 0.981


In [5]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
# Filter duration between 1 and 60 minutes
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
df["duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
df = df[(df["duration_min"] >= 1) & (df["duration_min"] <= 60)]

# Recast IDs to string
df["PULocationID"] = df["PULocationID"].astype(str)
df["DOLocationID"] = df["DOLocationID"].astype(str)

# Create list of dictionaries
dicts = df[["PULocationID", "DOLocationID"]].to_dict(orient="records")

# Apply DictVectorizer
dv = DictVectorizer()
X = dv.fit_transform(dicts)

# Dimensionality of the matrix
print("Shape of the feature matrix:", X.shape)

Shape of the feature matrix: (3009173, 515)


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# y is the target variable
y = df["duration_min"].values

# Train linear regression model
lr = LinearRegression()
lr.fit(X, y)

# Predict on training set
y_pred = lr.predict(X)

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("Training RMSE:", round(rmse, 2))


Training RMSE: 7.65


In [7]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the dataset
df = pd.read_parquet("./yellow_tripdata_2023-02.parquet")

# Preprocess durations
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
df["duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
df = df[(df["duration_min"] >= 1) & (df["duration_min"] <= 60)]

# One-hot encode pickup/dropoff IDs
df["PULocationID"] = df["PULocationID"].astype(str)
df["DOLocationID"] = df["DOLocationID"].astype(str)
dicts = df[["PULocationID", "DOLocationID"]].to_dict(orient="records")

# Vectorize
dv = DictVectorizer()
X = dv.fit_transform(dicts)
y = df["duration_min"].values

# Train Linear Regression
lr = LinearRegression()
lr.fit(X, y)

# Predict and calculate RMSE
y_pred = lr.predict(X)
rmse = mean_squared_error(y, y_pred, squared=False)
print("RMSE on training data:", round(rmse, 2))


RMSE on training data: 7.78
